<a href="https://colab.research.google.com/github/PearlSikka/language-ninja/blob/master/Sentiment_Analysis_using_LSTM_on_Amazon_food_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pathlib

# Upload the API token.
def get_kaggle():
  try:
    import kaggle
    return kaggle
  except OSError:
    pass

  token_file = pathlib.Path("~/.kaggle/kaggle.json").expanduser()
  token_file.parent.mkdir(exist_ok=True, parents=True)

  try:
    from google.colab import files
  except ImportError:
    raise ValueError("Could not find kaggle token.")

  uploaded = files.upload()
  token_content = uploaded.get('kaggle.json', None)
  if token_content:
    token_file.write_bytes(token_content)
    token_file.chmod(0o600)
  else:
    raise ValueError('Need a file named "kaggle.json"')
  
  import kaggle
  return kaggle


kaggle = get_kaggle()

In [ ]:
!kaggle datasets download -d snap/amazon-fine-food-reviews 

In [ ]:
!unzip amazon-fine-food-reviews.zip -d train

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
data=pd.read_csv('/content/train/Reviews.csv')

In [ ]:
data.head()

In [ ]:
data.describe

In [ ]:
data=data[['Text','Score']]

In [ ]:
data.head()

In [ ]:
data=data[data.Score !=3]    

Removing neutral comments

In [ ]:
data.isnull().any()

In [ ]:
rename_dict={1:0,2:0,4:1,5:1}

In [ ]:
rename_dict

In [ ]:
data =data.replace({"Score": rename_dict})

In [ ]:
data.head()

In [ ]:
#data['Text']=data['Text'].apply(lambda x: x.lower())
#data.head()

In [ ]:
#data['Text'].head(20)

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

from nltk.stem import PorterStemmer 

stemmer=PorterStemmer()

In [ ]:
def clean_word(text):
  split_sent= text.split()
  cleaned_Word=" ".join(stemmer.stem(i) for i in split_sent if 
                        i not in stopwords.words('english'))
  return cleaned_Word

In [ ]:
#data['Text']=data['Text'].apply(lambda x: clean_word(x))

In [ ]:
data.shape

In [ ]:
#data.head()

data1=data[data['Score']==0][:4000]

In [ ]:
data1=data1.append(data[data['Score']==1][:4000])

In [ ]:
data1.shape

In [ ]:
pos_cnt=data1[data1.Score==1]
neg_cnt=data1[data1.Score==0]

print(pos_cnt.shape)
print(neg_cnt.shape)

In [ ]:
data1.head()

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer=Tokenizer(num_words=1000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=' ')

In [ ]:
tokenizer.fit_on_texts(data1['Text'])

In [ ]:
print(tokenizer.word_index)

In [ ]:
print(len(tokenizer.word_index))

In [ ]:
sequences=tokenizer.texts_to_sequences(data1['Text'].values)

In [ ]:
print(sequences)

In [ ]:
print(sequences[0:10])

In [ ]:
padded=pad_sequences(sequences,maxlen=40,padding='post')

In [ ]:
print(padded[0:10])

In [ ]:
padded.shape


In [ ]:
X=padded

In [ ]:
from tensorflow.keras.models import Sequential
from keras.layers import Dense,SpatialDropout1D,LSTM,Embedding


embed_dim = 10
lstm_out = 100

model = Sequential()
model.add(Embedding(1000, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
Y=data1['Score'].values

In [ ]:
Y.shape

In [ ]:
X.shape

In [ ]:
#train_X,train_Y,test_X,test_Y= train_test_split(X,Y,test_size=0.25)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)


In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
model.fit(X_train,Y_train,batch_size=32,epochs=4)

In [ ]:
validation_size = 1000

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = 32)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

In [ ]:
twt = ['not tasty']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, 40, dtype='int32', value=0)
#print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
print(int(sentiment))




#print(np.argmax(sentiment))

if((sentiment) == 0):
    print("negative")
elif ((sentiment) == 1):
    print("positive")